# 7c – Capacity‑weighted distance minimisation

Minimise **Σ distance ÷ capacity** to balance geographic proximity with hospital size.  
Solved as a *linear‑assignment* problem (Hungarian algorithm).

In [ ]:

import math, numpy as np, pandas as pd
from scipy.optimize import linear_sum_assignment
from pathlib import Path

DATA_DIR = Path('.')
ACUTE_CSV = DATA_DIR / 'NHS_SW_Acute_Hospitals_enriched.csv'
CDC_CSV   = DATA_DIR / 'NHS_SW_Community_Diagnostic_Centres_enriched.csv'
CH_CSV    = DATA_DIR / 'NHS_SW_Community_Hospitals_enriched.csv'

acute = pd.read_csv(ACUTE_CSV)
cdc   = pd.read_csv(CDC_CSV)
ch    = pd.read_csv(CH_CSV)

# Ensure there is a 'capacity' column in the acute table
if 'capacity' not in acute.columns:
    acute['capacity'] = 1  # placeholder, set equal if unknown

R_EARTH = 6_371.0  # km
def haversine(lat1, lon1, lat2, lon2):
    φ1, λ1, φ2, λ2 = map(math.radians, (lat1, lon1, lat2, lon2))
    dφ, dλ = φ2 - φ1, λ2 - λ1
    a = math.sin(dφ/2)**2 + math.cos(φ1)*math.cos(φ2)*math.sin(dλ/2)**2
    return 2 * R_EARTH * math.atan2(math.sqrt(a), math.sqrt(1 - a))

spokes = pd.concat([cdc, ch], ignore_index=True)

cost = np.zeros((len(spokes), len(acute)))
for i, spk in spokes.iterrows():
    for j, hub in acute.iterrows():
        cost[i, j] = haversine(spk.latitude, spk.longitude,
                               hub.latitude, hub.longitude) / hub.capacity

row_ind, col_ind = linear_sum_assignment(cost)

spokes['nearest_acute'] = acute.loc[col_ind, 'Name'].values
spokes['weighted_dist'] = cost[row_ind, col_ind]

spokes.head()
